In [ ]:
import onnx

import tvm
from tvm import relay

onnx_model_path = '/home/pre-learn/onnx_runtime/model.onnx'

In [ ]:
model = onnx.load_model(onnx_model_path)

In [ ]:
import onnxruntime as onnxrt

sess_options = onnxrt.SessionOptions()
sess = onnxrt.InferenceSession(
        model.SerializeToString(),
        sess_options=sess_options,
        providers=onnxrt.get_available_providers(),
    )
meta = sess.get_modelmeta()

input_name = sess.get_inputs()[0].name
# shape_list = [(input_name, sess.get_inputs()[0].shape)]
shape = sess.get_inputs()[0].shape
shape[0] = 1
shape_list = {input_name:shape}

In [52]:
print(shape_list)
mod,params = relay.frontend.from_onnx(model,shape_list)

{'conv2d_input': [1, 28, 28, 1]}


In [ ]:
mod

In [48]:
with tvm.transform.PassContext(opt_level=3):
    intrp = relay.build_module.create_executor("graph", mod, tvm.cpu(0), target='llvm')

In [ ]:
import numpy as np
dtype = "float32"

out = intrp.evaluate()(tvm.nd.array(np.random.uniform(size=shape).astype(dtype)),**params).asnumpy()

In [53]:
target = 'llvm'
target_host = target
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, target_host=target_host,params=params)

In [54]:
import tvm.contrib.graph_executor as runtime

test_data = np.random.uniform(size=shape).astype(dtype)
ctx = tvm.cpu(0)
m = runtime.GraphModule(lib['default'](ctx))

def bench_tvm():
    m.set_input(input_name, tvm.nd.array(test_data))
    m.run()

%timeit bench_tvm()

116 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [55]:

def bench_ort():
    sess.run(None, {input_name: test_data})

%timeit bench_ort()

147 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
